In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from numpy.random import randn
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import KFold;
from sklearn import preprocessing
import math
from sklearn.metrics import mean_squared_error


C:\Users\rock\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Logistic -- linear model

### Train the model

In [2]:
train = pd.read_csv("train.csv")

In [3]:
train["area"] = train["slotheight"]*train["slotwidth"] 

feature = ["weekday","hour","advertiser","adexchange","area","slotvisibility","slotformat",
          "slotprice"]
X_train = train[feature]
Y_train = train["click"]
X_train.head()

,weekday,hour,advertiser,adexchange,area,slotvisibility,slotformat,slotprice
0,1,14,1458,2,28080,1,0,5
1,2,20,3476,2,62500,1,0,5
2,5,9,1458,1,94080,0,1,0
3,2,20,3427,2,65520,0,0,162
4,6,9,3476,1,85500,0,1,0


In [4]:
X_train.loc[ X_train['slotprice'] <= 0, 'slotprice'] = 0
X_train.loc[(X_train['slotprice'] > 0) & (X_train['slotprice'] <= 10), 'slotprice'] = 1
X_train.loc[(X_train['slotprice'] > 10) & (X_train['slotprice'] <= 50), 'slotprice'] = 2
X_train.loc[(X_train['slotprice'] > 50) & (X_train['slotprice'] <= 100), 'slotprice'] = 3
X_train.loc[(X_train['slotprice'] > 100), 'slotprice'] = 4


C:\Users\rock\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [5]:
X_train.area.value_counts().sort_index()
X_train.loc[ X_train['area'] <= 20000, 'area'] = 0
X_train.loc[(X_train['area'] > 20000) & (X_train['area'] <=40000 ), 'area'] = 1
X_train.loc[(X_train['area'] > 40000) & (X_train['area'] <= 60000), 'area'] = 2
X_train.loc[(X_train['area'] > 60000) & (X_train['area'] <= 80000), 'area'] = 3
X_train.loc[(X_train['area'] > 80000) & (X_train['area'] <= 100000), 'area'] = 4
X_train.loc[(X_train['area'] > 100000), 'area'] = 5


C:\Users\rock\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [6]:
X_train.head()

,weekday,hour,advertiser,adexchange,area,slotvisibility,slotformat,slotprice
0,1,14,1458,2,1,1,0,1
1,2,20,3476,2,3,1,0,1
2,5,9,1458,1,4,0,1,0
3,2,20,3427,2,3,0,0,4
4,6,9,3476,1,4,0,1,0


In [7]:
X_train["advertiser"]=X_train.advertiser.astype("str")
X_train["weekday"]=X_train.weekday.astype("str")
X_train["adexchange"]=X_train.adexchange.astype("str")
X_train["slotvisibility"]=X_train.slotvisibility.astype("str")
X_train["slotformat"]=X_train.slotformat.astype("str")
X_train["hour"]=X_train.hour.astype("str")


X_train=pd.get_dummies(X_train)

C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [8]:
logreg = LogisticRegression(penalty='l2', dual=False, tol=0.0001)
logreg.fit(X_train, Y_train)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

99.920000000000002

In [9]:
Y_pred = logreg.predict_proba(X_train)
Y_pred_log = Y_pred[:,1]
Y_pred_log

array([ 0.00079713,  0.00066784,  0.00086262, ...,  0.00034722,
        0.00071372,  0.00091048])

In [10]:
import math
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(Y_pred_log ,Y_train)
RMSE = math.sqrt(MSE)
print("RMSE: " + str(RMSE) )

RMSE: 0.027433793818018283


### Evaluation Matrix --Logistic
#### base bid = 100

In [11]:
train["pCTR_log"] = Y_pred_log

avgCTR = Y_pred_log.mean()

param_log= Y_pred_log/avgCTR

basebid = train.bidprice.mean()
# budget = 25000
train['new_bidprice_log'] = 100*param_log

statistic=pd.DataFrame()
new_train = train[train["new_bidprice_log"]>=train.bidprice]
new_train["total_cost"] = new_train["payprice"].cumsum()
new_train = new_train[new_train["total_cost"]/1000<=25000]
statistic["Click"] = [new_train[new_train.click==1].click.sum()]
statistic["Spend"] = new_train.payprice.sum()/1000
statistic["CTR%"] = statistic["Click"]/new_train.shape[0]
statistic["CPC"] = statistic["Spend"]/statistic["Click"]

statistic_2=pd.DataFrame()
budget_2 = new_train[new_train["total_cost"]/1000<=25000*1/2]
statistic_2["Click"] = [budget_2[budget_2.click==1].click.sum()]
statistic_2["Spend"] = budget_2.payprice.sum()/1000
statistic_2["CTR%"] = statistic_2["Click"]/budget_2.shape[0]
statistic_2["CPC"] = statistic_2["Spend"]/statistic_2["Click"]

statistic_4=pd.DataFrame()
budget_4 = new_train[new_train["total_cost"]/1000<=25000/4]
statistic_4["Click"] = [budget_4[budget_4.click==1].click.sum()]
statistic_4["Spend"] = budget_4.payprice.sum()/1000
statistic_4["CTR%"] = statistic_4["Click"]/budget_4.shape[0]
statistic_4["CPC"] = statistic_4["Spend"]/statistic_4["Click"]

statistic_8=pd.DataFrame()
budget_8 = new_train[new_train["total_cost"]/1000<=25000*1/8]
statistic_8["Click"] = [budget_8[budget_8.click==1].click.sum()]
statistic_8["Spend"] = budget_8.payprice.sum()/1000
statistic_8["CTR%"] = statistic_8["Click"]/budget_8.shape[0]
statistic_8["CPC"] = statistic_8["Spend"]/statistic_8["Click"]

statistic_16=pd.DataFrame()
budget_16 = new_train[new_train["total_cost"]/1000<=25000*1/16]
statistic_16["Click"] = [budget_16[budget_16.click==1].click.sum()]
statistic_16["Spend"] = budget_16.payprice.sum()/1000
statistic_16["CTR%"] = statistic_16["Click"]/budget_16.shape[0]
statistic_16["CPC"] = statistic_16["Spend"]/statistic_16["Click"]


statistic_one = statistic.append(statistic_2)
statistic_two = statistic_one.append(statistic_4)
statistic_3 = statistic_two.append(statistic_8)
statistic_full = statistic_3.append(statistic_16)

statistic_full.index = ["full","1/2","1/4","1/8","1/16"]
statistic_full

C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPC
full,361,5370.389,0.005640,14.876424
1/2,361,5370.389,0.005640,14.876424
1/4,361,5370.389,0.005640,14.876424
1/8,207,3124.905,0.005561,15.096159
1/16,114,1562.362,0.006166,13.704930


In [12]:
train.head()

,click,weekday,hour,bidid,logtype,userid,useragent,IP,region,city,...,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,area,pCTR_log,new_bidprice_log
0,0,1,14,fdfae6789b787899f1b875de3ab8b21a,1,u_Vh1OPkFv3q5CFdR,windows_ie,180.107.112.*,80,85,...,5,612599432d200b093719dd1f372f7a30,300,54,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10063",28080,0.000797,105.525350
1,0,2,20,372883147cdefbc495fb5082f79dde9b,1,u_VhkRL6dCOTLsq-c,windows_chrome,125.120.199.*,94,95,...,5,8dff45ed862a740986dbe688aafee7e5,249,42,15b749127478946f161a54dc0dad27c8,3476,"10063,10111",62500,0.000668,88.409430
2,0,5,9,2912cb5947e1fad450e2473c2f61ef75,1,u_VhkyORMrD8TP3GL,windows_ie,120.196.64.*,216,217,...,0,77819d3e0b3467fe5c7b16d68ad923a1,300,76,bebefa5efe83beee17a3d245e7c5085b,1458,10063,94080,0.000863,114.194455
3,0,2,20,d7b7d61d1e50e1d27d210928d28a9b61,1,u_VhkyLRqVL2a4eWL,windows_ie,219.131.122.*,216,221,...,162,7184c9560e68e977187e67e45a4f3198,238,182,0f951a030abdaedd733ee8d114ce2944,3427,"10077,10111",65520,0.000928,122.893224
4,0,6,9,657d317dc4045bb79c2652abbea07530,1,u_Vh1zOJuQOtTCBgL,windows_chrome,123.190.78.*,40,48,...,0,c938195f9e404b4f38c7e71bf50263e5,238,34,a3becb967c015e9acb5e72f22b100ef8,3476,"10031,13042,10006,10057,13866,10063,14273,1005...",85500,0.000426,56.451560


### Validation

In [13]:
validation = pd.read_csv("validation.csv")

In [14]:
validation["area"] = validation["slotheight"]*validation["slotwidth"] 

feature = ["weekday","hour","advertiser","adexchange","area","slotvisibility","slotformat",
          "slotprice"]
X_validation = validation[feature]
Y_validation = validation["click"]
X_validation.head()

,weekday,hour,advertiser,adexchange,area,slotvisibility,slotformat,slotprice
0,5,14,3427,1,75000,0,1,0
1,3,14,2997,null,16000,FirstView,Na,118
2,1,13,3427,3,90000,0,0,70
3,5,18,3427,3,90000,0,0,70
4,5,14,3386,2,94080,2,0,5


In [15]:
X_validation.loc[ X_validation['slotprice'] <= 0, 'slotprice'] = 0
X_validation.loc[(X_validation['slotprice'] > 0) & (X_validation['slotprice'] <= 10), 'slotprice'] = 1
X_validation.loc[(X_validation['slotprice'] > 10) & (X_validation['slotprice'] <= 50), 'slotprice'] = 2
X_validation.loc[(X_validation['slotprice'] > 50) & (X_validation['slotprice'] <= 100), 'slotprice'] = 3
X_validation.loc[(X_validation['slotprice'] > 100), 'slotprice'] = 4

X_validation.area.value_counts().sort_index()
X_validation.loc[ X_validation['area'] <= 20000, 'area'] = 0
X_validation.loc[(X_validation['area'] > 20000) & (X_validation['area'] <=40000 ), 'area'] = 1
X_validation.loc[(X_validation['area'] > 40000) & (X_validation['area'] <= 60000), 'area'] = 2
X_validation.loc[(X_validation['area'] > 60000) & (X_validation['area'] <= 80000), 'area'] = 3
X_validation.loc[(X_validation['area'] > 80000) & (X_validation['area'] <= 100000), 'area'] = 4
X_validation.loc[(X_validation['area'] > 100000), 'area'] = 5


C:\Users\rock\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [16]:
X_validation["advertiser"]=X_validation.advertiser.astype("str")
X_validation["weekday"]=X_validation.weekday.astype("str")
X_validation["adexchange"]=X_validation.adexchange.astype("str")
X_validation["slotvisibility"]=X_validation.slotvisibility.astype("str")
X_validation["slotformat"]=X_validation.slotformat.astype("str")
X_validation["hour"]=X_validation.hour.astype("str")

# X_validation["slotprice"]=X_validation.slotprice.astype("str")
# X_validation["area"]=X_validation.area.astype("str")

X_validation=pd.get_dummies(X_validation)

C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [17]:
Y_pred_validation = logreg.predict_proba(X_validation)
Y_pred_validation_log = Y_pred_validation[:,1]
Y_pred_validation_log

array([ 0.00063285,  0.00572677,  0.0008557 , ...,  0.00035328,
        0.00025479,  0.00114611])

In [18]:
MSE = mean_squared_error(Y_pred_validation_log,Y_validation)
RMSE = math.sqrt(MSE)
print("RMSE: " + str(RMSE) )

RMSE: 0.027433630306692516


In [19]:
validation["pCTR_log"] = Y_pred_validation_log

# avgCTR = (train[train.click==1].groupby("advertiser").size().values/train.groupby("advertiser").size().values).mean()
avgCTR = Y_pred_validation_log.mean()

param_validation_log= Y_pred_validation_log/avgCTR
param_validation_log

array([ 0.84192958,  7.61875047,  1.13839622, ...,  0.47000023,
        0.33896518,  1.52475801])

### Performance matrix
#### base bid = 100

In [20]:
basebid = train.bidprice.mean()
# budget = 25000
validation['new_bidprice_log'] = 100*param_validation_log

statistic=pd.DataFrame()
new_validation = validation[validation["new_bidprice_log"]>=validation.payprice]
new_validation["total_cost"] = new_validation["payprice"].cumsum()
new_validation = new_validation[new_validation["total_cost"]/1000<=25000]
statistic["Click"] = [new_validation[new_validation.click==1].click.sum()]
statistic["Spend"] = new_validation.payprice.sum()/1000
statistic["CTR%"] = statistic["Click"]/new_validation.shape[0]
statistic["CPC"] = statistic["Spend"]/statistic["Click"]

statistic_4=pd.DataFrame()
budget_4 = new_validation[new_validation["total_cost"]/1000<=25000*1/4]
statistic_4["Click"] = [budget_4[budget_4.click==1].click.sum()]
statistic_4["Spend"] = budget_4.payprice.sum()/1000
statistic_4["CTR%"] = statistic_4["Click"]/budget_4.shape[0]
statistic_4["CPC"] = statistic_4["Spend"]/statistic_4["Click"]

statistic_2=pd.DataFrame()
budget_2 = new_validation[new_validation["total_cost"]/1000<=25000*1/2]
statistic_2["Click"] = [budget_2[budget_2.click==1].click.sum()]
statistic_2["Spend"] = budget_2.payprice.sum()/1000
statistic_2["CTR%"] = statistic_2["Click"]/budget_2.shape[0]
statistic_2["CPC"] = statistic_2["Spend"]/statistic_2["Click"]

statistic_16=pd.DataFrame()
budget_16 = new_validation[new_validation["total_cost"]/1000<=25000*1/16]
statistic_16["Click"] = [budget_16[budget_16.click==1].click.sum()]
statistic_16["Spend"] = budget_16.payprice.sum()/1000
statistic_16["CTR%"] = statistic_16["Click"]/budget_16.shape[0]
statistic_16["CPC"] = statistic_16["Spend"]/statistic_16["Click"]

statistic_8=pd.DataFrame()
budget_8 = new_validation[new_validation["total_cost"]/1000<=25000*1/8]
statistic_8["Click"] = [budget_8[budget_8.click==1].click.sum()]
statistic_8["Spend"] = budget_8.payprice.sum()/1000
statistic_8["CTR%"] = statistic_8["Click"]/budget_8.shape[0]
statistic_8["CPC"] = statistic_8["Spend"]/statistic_8["Click"]

statistic_one = statistic.append(statistic_2)
statistic_two = statistic_one.append(statistic_4)
statistic_3 = statistic_two.append(statistic_8)
statistic_full = statistic_3.append(statistic_16)

statistic_full.index = ["full","1/2","1/4","1/8","1/16"]
statistic_full

C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPC
full,138,9735.674,0.000707,70.548362
1/2,138,9735.674,0.000707,70.548362
1/4,89,6249.985,0.000709,70.224551
1/8,54,3124.967,0.000861,57.869759
1/16,30,1562.456,0.000957,52.081867


#### base bid = 272

In [21]:
validation['new_bidprice_log'] = 272*param_validation_log

statistic=pd.DataFrame()
new_validation = validation[validation["new_bidprice_log"]>=validation.payprice]
new_validation["total_cost"] = new_validation["payprice"].cumsum()
new_validation = new_validation[new_validation["total_cost"]/1000<=25000]
statistic["Click"] = [new_validation[new_validation.click==1].click.sum()]
statistic["Spend"] = new_validation.payprice.sum()/1000
statistic["CTR%"] = statistic["Click"]/new_validation.shape[0]
statistic["CPC"] = statistic["Spend"]/statistic["Click"]

statistic_4=pd.DataFrame()
budget_4 = new_validation[new_validation["total_cost"]/1000<=25000*1/4]
statistic_4["Click"] = [budget_4[budget_4.click==1].click.sum()]
statistic_4["Spend"] = budget_4.payprice.sum()/1000
statistic_4["CTR%"] = statistic_4["Click"]/budget_4.shape[0]
statistic_4["CPC"] = statistic_4["Spend"]/statistic_4["Click"]

statistic_2=pd.DataFrame()
budget_2 = new_validation[new_validation["total_cost"]/1000<=25000*1/2]
statistic_2["Click"] = [budget_2[budget_2.click==1].click.sum()]
statistic_2["Spend"] = budget_2.payprice.sum()/1000
statistic_2["CTR%"] = statistic_2["Click"]/budget_2.shape[0]
statistic_2["CPC"] = statistic_2["Spend"]/statistic_2["Click"]

statistic_16=pd.DataFrame()
budget_16 = new_validation[new_validation["total_cost"]/1000<=25000*1/16]
statistic_16["Click"] = [budget_16[budget_16.click==1].click.sum()]
statistic_16["Spend"] = budget_16.payprice.sum()/1000
statistic_16["CTR%"] = statistic_16["Click"]/budget_16.shape[0]
statistic_16["CPC"] = statistic_16["Spend"]/statistic_16["Click"]

statistic_8=pd.DataFrame()
budget_8 = new_validation[new_validation["total_cost"]/1000<=25000*1/8]
statistic_8["Click"] = [budget_8[budget_8.click==1].click.sum()]
statistic_8["Spend"] = budget_8.payprice.sum()/1000
statistic_8["CTR%"] = statistic_8["Click"]/budget_8.shape[0]
statistic_8["CPC"] = statistic_8["Spend"]/statistic_8["Click"]

statistic_one = statistic.append(statistic_2)
statistic_two = statistic_one.append(statistic_4)
statistic_3 = statistic_two.append(statistic_8)
statistic_full = statistic_3.append(statistic_16)

statistic_full.index = ["full","1/2","1/4","1/8","1/16"]
statistic_full

C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPC
full,210,19192.627,0.000767,91.393462
1/2,134,12499.955,0.000751,93.283246
1/4,81,6249.987,0.000908,77.160333
1/8,46,3124.988,0.001032,67.934522
1/16,24,1562.493,0.001082,65.103875


In [22]:
validation.head()

,click,weekday,hour,bidid,logtype,userid,useragent,IP,region,city,...,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,area,pCTR_log,new_bidprice_log
0,0,5,14,91c6a6b9e90c0f54d3230815a5a3e22e,1,u_Vhk7C5STO8TZ3s8,windows_ie,14.122.240.*,216,232,...,0,449a22cd91d9042eda3d3a1b89a22ea8,227,102,0f951a030abdaedd733ee8d114ce2944,3427,"10006,10063,10059,10111",75000,0.000633,229.004846
1,0,3,14,24b3621ad3b063b6c09c541781d534b3,1,u_DANDbCAksMy,android_safari,116.22.55.*,216,217,...,118,11908,277,118,null,2997,null,16000,0.005727,2072.300129
2,0,1,13,6e7bce0a9ad42f479827d37bf9618a9d,1,u_VhkxPnKHPo1H3Vb,windows_ie,60.215.242.*,146,149,...,70,d01411218cc79bc49d2a4078c4093b76,241,77,b2e35064f3549d447edbbdfb1f707c8c,3427,null,90000,0.000856,309.643773
3,0,5,18,6873feca381b8293bf418ba8408a014c,1,u_Vh5zL3B83ln9Xbl,windows_chrome,27.193.87.*,146,148,...,70,c46090c887c257b61ab1fa11baee91d8,241,77,0f951a030abdaedd733ee8d114ce2944,3427,null,90000,0.000970,350.846358
4,0,5,14,7e6430333c552cf0b32bb21dc792d2df,1,u_VhKPOr5wOt9HQsj,windows_ie,183.228.214.*,275,275,...,5,ff5123fb9333ca095034c62fdaaf51aa,300,178,361e128affece850342293213691a043,3386,null,94080,0.000352,127.211864


## Gradient

In [23]:
train["area"] = train["slotheight"]*train["slotwidth"] 

city = pd.DataFrame(train.city.value_counts()/train.shape[0]).index
frequent = pd.DataFrame(train.city.value_counts()/train.shape[0]).values
r = pd.DataFrame()
r["city"] = city
r["frequent"] = frequent
train=pd.merge(train,r,on='city',how='left') 

feature = ["weekday","hour","advertiser","adexchange","area","slotvisibility","slotformat","frequent",
          "slotprice"]
X_train = train[feature]
Y_train = train["click"]

X_train.head()

,weekday,hour,advertiser,adexchange,area,slotvisibility,slotformat,frequent,slotprice
0,1,14,1458,2,28080,1,0,0.019417,5
1,2,20,3476,2,62500,1,0,0.013130,5
2,5,9,1458,1,94080,0,1,0.039022,0
3,2,20,3427,2,65520,0,0,0.004477,162
4,6,9,3476,1,85500,0,1,0.001714,0


In [24]:
np.random.seed(12)

# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()

X_train["slotvisibility"] = label_encoder.fit_transform(X_train["slotvisibility"])
X_train["slotformat"] = label_encoder.fit_transform(X_train["slotformat"])
X_train["adexchange"] = label_encoder.fit_transform(X_train["adexchange"])

X_train["advertiser"]=X_train.advertiser.astype("str")
X_train["weekday"]=X_train.weekday.astype("str")
X_train["hour"]=X_train.hour.astype("str")

X_train=pd.get_dummies(X_train)

C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

In [25]:
import xgboost as xgb
#model_xgb = xgb.XGBClassifier(n_estimators=500, max_depth=3, learning_rate=0.5)
model_xgb = xgb.XGBClassifier(n_estimators=300)
model_xgb.fit(X_train,Y_train)

GBR = GradientBoostingClassifier(n_estimators=300) #200
GBR.fit(X_train, Y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=300, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [26]:
from sklearn.linear_model import ElasticNet, LassoCV, LassoLarsCV
model_lasso = LassoCV(alphas = [1, 0.1, 0.01,0.001, 0.0005]).fit(X_train,Y_train)

In [27]:
XGB_prediction2D=model_xgb.predict_proba(X_train)
Y_pred_XGB = XGB_prediction2D[:,1]

Y_pred_lasso=model_lasso.predict(X_train)
Y_pred_ = GBR.predict_proba(X_train)   #
Y_pred_gbr1 = XGB_prediction2D[:,1]    #
Y_pred_gbr=0*Y_pred_lasso+0.5*Y_pred_XGB+0.5*Y_pred_gbr1
Y_pred_gbr

array([ 0.00086655,  0.00030427,  0.00111079, ...,  0.00025365,
        0.00039957,  0.00078972])

In [28]:
MSE = mean_squared_error(Y_pred_gbr,Y_train)
RMSE = math.sqrt(MSE)
print("RMSE: " + str(RMSE) )

RMSE: 0.027418830785208386


### Performance Matrix --- Train
#### Base bid = 100

In [29]:
train["pCTR_gbr"] = Y_pred_gbr

avgCTR = Y_pred_gbr.mean()

param_gbr= Y_pred_gbr/avgCTR

train['new_bidprice_gbr'] = 100*param_gbr

statistic=pd.DataFrame()
new_train = train[train["new_bidprice_gbr"]>=train.payprice]
new_train["total_cost"] = new_train["payprice"].cumsum()
new_train = new_train[new_train["total_cost"]/1000<=25000]
statistic["Click"] = [new_train[new_train.click==1].click.sum()]
statistic["Spend"] = new_train.payprice.sum()/1000
statistic["CTR%"] = statistic["Click"]/new_train.shape[0]
statistic["CPC"] = statistic["Spend"]/statistic["Click"]

statistic_4=pd.DataFrame()
budget_4 = new_train[new_train["total_cost"]/1000<=25000*1/4]
statistic_4["Click"] = [budget_4[budget_4.click==1].click.sum()]
statistic_4["Spend"] = budget_4.payprice.sum()/1000
statistic_4["CTR%"] = statistic_4["Click"]/budget_4.shape[0]
statistic_4["CPC"] = statistic_4["Spend"]/statistic_4["Click"]

statistic_2=pd.DataFrame()
budget_2 = new_train[new_train["total_cost"]/1000<=25000*1/2]
statistic_2["Click"] = [budget_2[budget_2.click==1].click.sum()]
statistic_2["Spend"] = budget_2.payprice.sum()/1000
statistic_2["CTR%"] = statistic_2["Click"]/budget_2.shape[0]
statistic_2["CPC"] = statistic_2["Spend"]/statistic_2["Click"]

statistic_16=pd.DataFrame()
budget_16 = new_train[new_train["total_cost"]/1000<=25000*1/16]
statistic_16["Click"] = [budget_16[budget_16.click==1].click.sum()]
statistic_16["Spend"] = budget_16.payprice.sum()/1000
statistic_16["CTR%"] = statistic_16["Click"]/budget_16.shape[0]
statistic_16["CPC"] = statistic_16["Spend"]/statistic_16["Click"]

statistic_8=pd.DataFrame()
budget_8 = new_train[new_train["total_cost"]/1000<=25000*1/8]
statistic_8["Click"] = [budget_8[budget_8.click==1].click.sum()]
statistic_8["Spend"] = budget_8.payprice.sum()/1000
statistic_8["CTR%"] = statistic_8["Click"]/budget_8.shape[0]
statistic_8["CPC"] = statistic_8["Spend"]/statistic_8["Click"]

statistic_one = statistic.append(statistic_2)
statistic_two = statistic_one.append(statistic_4)
statistic_3 = statistic_two.append(statistic_8)
statistic_full = statistic_3.append(statistic_16)

statistic_full.index = ["full","1/2","1/4","1/8","1/16"]
statistic_full

C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPC
full,445,24999.965,0.000912,56.179697
1/2,218,12499.957,0.000893,57.339252
1/4,100,6249.973,0.000818,62.499730
1/8,55,3124.995,0.000899,56.818091
1/16,24,1562.436,0.000786,65.101500


### Validation

In [30]:
validation["area"] = validation["slotheight"]*validation["slotwidth"] 

city = pd.DataFrame(validation.city.value_counts()/validation.shape[0]).index
frequent = pd.DataFrame(validation.city.value_counts()/validation.shape[0]).values
r = pd.DataFrame()
r["city"] = city
r["frequent"] = frequent
validation=pd.merge(validation,r,on='city',how='left') 

feature = ["weekday","hour","advertiser","adexchange","area","slotvisibility","slotformat","frequent",
          "slotprice"]
X_validation = validation[feature]
Y_validation = validation["click"]

X_validation.head()

,weekday,hour,advertiser,adexchange,area,slotvisibility,slotformat,frequent,slotprice
0,5,14,3427,1,75000,0,1,0.002068,0
1,3,14,2997,null,16000,FirstView,Na,0.039229,118
2,1,13,3427,3,90000,0,0,0.003676,70
3,5,18,3427,3,90000,0,0,0.010175,70
4,5,14,3386,2,94080,2,0,0.021628,5


In [31]:
np.random.seed(12)

# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()

X_validation["slotvisibility"] = label_encoder.fit_transform(X_validation["slotvisibility"])
X_validation["slotformat"] = label_encoder.fit_transform(X_validation["slotformat"])
X_validation["adexchange"] = label_encoder.fit_transform(X_validation["adexchange"])


X_validation["advertiser"]=X_validation.advertiser.astype("str")
X_validation["weekday"]=X_validation.weekday.astype("str")
X_validation["hour"]=X_validation.hour.astype("str")

X_validation=pd.get_dummies(X_validation)

C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

In [80]:
Y_pred_validation_gbr1 = GBR.predict_proba(X_validation)
Y_pred_validation_gbr1 = Y_pred_validation_gbr1 [:,1]


XGB_prediction2D=model_xgb.predict_proba(X_validation)
Y_pred_XGB = XGB_prediction2D[:,1]

Y_pred_lasso=model_lasso.predict(X_validation)
Y_pred_validation_gbr=0.25*Y_pred_lasso+0.25*Y_pred_XGB+0.5*Y_pred_validation_gbr1
#Y_pred_validation_gbr=0.25*Y_pred_lasso+0.25*Y_pred_XGB+0.5*Y_pred_validation_gbr1
#Y_pred_validation_gbr=0.40*Y_pred_lasso+0.3*Y_pred_XGB+0.3*Y_pred_validation_gbr1

MSE = mean_squared_error(Y_pred_validation_gbr,Y_validation)
RMSE = math.sqrt(MSE)
print("RMSE: " + str(RMSE) )

RMSE: 0.02746792239911175


### Performace Matrix--validaiton
#### Base bid = 100

In [81]:
validation["pCTR_gbr"] = Y_pred_validation_gbr

avgCTR = Y_pred_validation_gbr.mean()

param_validation_gbr= Y_pred_validation_gbr/avgCTR

validation['new_bidprice_gbr'] = 270*param_validation_gbr

statistic=pd.DataFrame()
new_validation = validation[validation["new_bidprice_gbr"]>=validation.payprice]
new_validation["total_cost"] = new_validation["payprice"].cumsum()
new_validation = new_validation[new_validation["total_cost"]/1000<=25000]
statistic["Click"] = [new_validation[new_validation.click==1].click.sum()]
statistic["Spend"] = new_validation.payprice.sum()/1000
statistic["CTR%"] = statistic["Click"]/new_validation.shape[0]
statistic["CPC"] = statistic["Spend"]/statistic["Click"]

statistic_4=pd.DataFrame()
budget_4 = new_validation[new_validation["total_cost"]/1000<=25000*1/4]
statistic_4["Click"] = [budget_4[budget_4.click==1].click.sum()]
statistic_4["Spend"] = budget_4.payprice.sum()/1000
statistic_4["CTR%"] = statistic_4["Click"]/budget_4.shape[0]
statistic_4["CPC"] = statistic_4["Spend"]/statistic_4["Click"]

statistic_2=pd.DataFrame()
budget_2 = new_validation[new_validation["total_cost"]/1000<=25000*1/2]
statistic_2["Click"] = [budget_2[budget_2.click==1].click.sum()]
statistic_2["Spend"] = budget_2.payprice.sum()/1000
statistic_2["CTR%"] = statistic_2["Click"]/budget_2.shape[0]
statistic_2["CPC"] = statistic_2["Spend"]/statistic_2["Click"]

statistic_16=pd.DataFrame()
budget_16 = new_validation[new_validation["total_cost"]/1000<=25000*1/16]
statistic_16["Click"] = [budget_16[budget_16.click==1].click.sum()]
statistic_16["Spend"] = budget_16.payprice.sum()/1000
statistic_16["CTR%"] = statistic_16["Click"]/budget_16.shape[0]
statistic_16["CPC"] = statistic_16["Spend"]/statistic_16["Click"]

statistic_8=pd.DataFrame()
budget_8 = new_validation[new_validation["total_cost"]/1000<=25000*1/8]
statistic_8["Click"] = [budget_8[budget_8.click==1].click.sum()]
statistic_8["Spend"] = budget_8.payprice.sum()/1000
statistic_8["CTR%"] = statistic_8["Click"]/budget_8.shape[0]
statistic_8["CPC"] = statistic_8["Spend"]/statistic_8["Click"]

statistic_one = statistic.append(statistic_2)
statistic_two = statistic_one.append(statistic_4)
statistic_3 = statistic_two.append(statistic_8)
statistic_full = statistic_3.append(statistic_16)

statistic_full.index = ["full","1/2","1/4","1/8","1/16"]
statistic_full

C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPC
full,210,20352.445,0.000744,96.916405
1/2,133,12499.865,0.000767,93.983947
1/4,75,6249.897,0.000866,83.331960
1/8,46,3124.828,0.001064,67.931043
1/16,23,1562.486,0.001067,67.934174


#### base price = 272

# Chien-Liang,Lin

In [83]:
validation['new_bidprice_gbr'] = 73*param_validation_gbr

statistic=pd.DataFrame()
new_validation = validation[validation["new_bidprice_gbr"]>=validation.payprice]
new_validation["total_cost"] = new_validation["payprice"].cumsum()
budget_1 = new_validation[new_validation["total_cost"]/1000<=25000]
statistic["Click"] = [budget_1[budget_1.click==1].click.sum()]
statistic["Spend"] = budget_1.payprice.sum()/1000
statistic["CTR%"] = statistic["Click"]/budget_1.shape[0]*100
statistic["CPC"] = statistic["Spend"]/statistic["Click"]
statistic["CPM"] = statistic["Spend"] /budget_1.shape[0]* 1000

statistic_2=pd.DataFrame()
budget_2 = new_validation[new_validation["total_cost"]/1000<=25000*1/2]
statistic_2["Click"] = [budget_2[budget_2.click==1].click.sum()]
statistic_2["Spend"] = budget_2.payprice.sum()/1000
statistic_2["CTR%"] = statistic_2["Click"]/budget_2.shape[0]*100
statistic_2["CPC"] = statistic_2["Spend"]/statistic_2["Click"]
statistic_2["CPM"] = statistic_2["Spend"] /budget_2.shape[0]* 1000

statistic_4=pd.DataFrame()
budget_4 = new_validation[new_validation["total_cost"]/1000<=25000/4]
statistic_4["Click"] = [budget_4[budget_4.click==1].click.sum()]
statistic_4["Spend"] = budget_4.payprice.sum()/1000
statistic_4["CTR%"] = statistic_4["Click"]/budget_4.shape[0]*100
statistic_4["CPC"] = statistic_4["Spend"]/statistic_4["Click"]
statistic_4["CPM"] = statistic_4["Spend"] /budget_4.shape[0]* 1000

statistic_8=pd.DataFrame()
budget_8 = new_validation[new_validation["total_cost"]/1000<=25000*1/8]
statistic_8["Click"] = [budget_8[budget_8.click==1].click.sum()]
statistic_8["Spend"] = budget_8.payprice.sum()/1000
statistic_8["CTR%"] = statistic_8["Click"]/budget_8.shape[0]*100
statistic_8["CPC"] = statistic_8["Spend"]/statistic_8["Click"]
statistic_8["CPM"] = statistic_8["Spend"] /budget_8.shape[0]* 1000

statistic_16=pd.DataFrame()
budget_16 = new_validation[new_validation["total_cost"]/1000<=25000*1/16]
statistic_16["Click"] = [budget_16[budget_16.click==1].click.sum()]
statistic_16["Spend"] = budget_16.payprice.sum()/1000
statistic_16["CTR%"] = statistic_16["Click"]/budget_16.shape[0]*100
statistic_16["CPC"] = statistic_16["Spend"]/statistic_16["Click"]
statistic_16["CPM"] = statistic_16["Spend"] /budget_16.shape[0]* 1000


statistic_one = statistic.append(statistic_2)
statistic_two = statistic_one.append(statistic_4)
statistic_3 = statistic_two.append(statistic_8)
statistic_full_val_mix = statistic_3.append(statistic_16)

statistic_full_val_mix.index = ["full","1/2","1/4 (6250)","1/8","1/16"]
statistic_full_val_mix

C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPC,CPM
full,118,6579.546,0.076984,55.758864,42.925293
1/2,118,6579.546,0.076984,55.758864,42.925293
1/4 (6250),111,6249.995,0.076168,56.306261,42.887203
1/8,62,3124.976,0.084849,50.402839,42.766296
1/16,36,1562.497,0.098256,43.402694,42.645733


# lasso *(1/4) + XGBoost *(1/4) + gbr* (1/2)　

# XGBoost

In [79]:
Y_pred_validation = model_xgb.predict_proba(X_validation)
Y_pred_validation_xgb = Y_pred_validation[:,1]
validation["pCTR_xgb"] = Y_pred_validation_xgb

# avgCTR = (train[train.click==1].groupby("advertiser").size().values/train.groupby("advertiser").size().values).mean()
avgCTR = Y_pred_validation_xgb.mean()

param_validation_xgb= Y_pred_validation_xgb/avgCTR

basebid = train.bidprice.mean()
# budget = 25000
validation['new_bidprice_xgb'] = 70*param_validation_xgb

statistic=pd.DataFrame()
new_validation = validation[validation["new_bidprice_xgb"]>=validation.payprice]
new_validation["total_cost"] = new_validation["payprice"].cumsum()
budget_1 = new_validation[new_validation["total_cost"]/1000<=25000]
statistic["Click"] = [budget_1[budget_1.click==1].click.sum()]
statistic["Spend"] = budget_1.payprice.sum()/1000
statistic["CTR%"] = statistic["Click"]/budget_1.shape[0]*100
statistic["CPC"] = statistic["Spend"]/statistic["Click"]
statistic["CPM"] = statistic["Spend"] /budget_1.shape[0]* 1000

statistic_2=pd.DataFrame()
budget_2 = new_validation[new_validation["total_cost"]/1000<=25000*1/2]
statistic_2["Click"] = [budget_2[budget_2.click==1].click.sum()]
statistic_2["Spend"] = budget_2.payprice.sum()/1000
statistic_2["CTR%"] = statistic_2["Click"]/budget_2.shape[0]*100
statistic_2["CPC"] = statistic_2["Spend"]/statistic_2["Click"]
statistic_2["CPM"] = statistic_2["Spend"] /budget_2.shape[0]* 1000

statistic_4=pd.DataFrame()
budget_4 = new_validation[new_validation["total_cost"]/1000<=25000/4]
statistic_4["Click"] = [budget_4[budget_4.click==1].click.sum()]
statistic_4["Spend"] = budget_4.payprice.sum()/1000
statistic_4["CTR%"] = statistic_4["Click"]/budget_4.shape[0]*100
statistic_4["CPC"] = statistic_4["Spend"]/statistic_4["Click"]
statistic_4["CPM"] = statistic_4["Spend"] /budget_4.shape[0]* 1000

statistic_8=pd.DataFrame()
budget_8 = new_validation[new_validation["total_cost"]/1000<=25000*1/8]
statistic_8["Click"] = [budget_8[budget_8.click==1].click.sum()]
statistic_8["Spend"] = budget_8.payprice.sum()/1000
statistic_8["CTR%"] = statistic_8["Click"]/budget_8.shape[0]*100
statistic_8["CPC"] = statistic_8["Spend"]/statistic_8["Click"]
statistic_8["CPM"] = statistic_8["Spend"] /budget_8.shape[0]* 1000

statistic_16=pd.DataFrame()
budget_16 = new_validation[new_validation["total_cost"]/1000<=25000*1/16]
statistic_16["Click"] = [budget_16[budget_16.click==1].click.sum()]
statistic_16["Spend"] = budget_16.payprice.sum()/1000
statistic_16["CTR%"] = statistic_16["Click"]/budget_16.shape[0]*100
statistic_16["CPC"] = statistic_16["Spend"]/statistic_16["Click"]
statistic_16["CPM"] = statistic_16["Spend"] /budget_16.shape[0]* 1000


statistic_one = statistic.append(statistic_2)
statistic_two = statistic_one.append(statistic_4)
statistic_3 = statistic_two.append(statistic_8)

statistic_full_val_xgb = statistic_3.append(statistic_16)

statistic_full_val_xgb.index = ["full","1/2","1/4 (6250)","1/8","1/16"]
statistic_full_val_xgb

C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPC,CPM
full,116,6093.271,0.084943,52.528198,44.619081
1/2,116,6093.271,0.084943,52.528198,44.619081
1/4 (6250),116,6093.271,0.084943,52.528198,44.619081
1/8,63,3124.961,0.089543,49.602556,44.415780
1/16,36,1562.480,0.102305,43.402222,44.402512


# Lasso

In [36]:

Y_pred_validation_lasso=model_lasso.predict(X_validation)
validation["pCTR_Lasso"] = Y_pred_validation_lasso

# avgCTR = (train[train.click==1].groupby("advertiser").size().values/train.groupby("advertiser").size().values).mean()
avgCTR = Y_pred_validation_lasso.mean()

param_validation_lasso= Y_pred_validation_lasso/avgCTR

basebid = train.bidprice.mean()
# budget = 25000
validation['new_bidprice_lasso'] = 80*param_validation_lasso

statistic=pd.DataFrame()
new_validation = validation[validation["new_bidprice_lasso"]>=validation.payprice]
new_validation["total_cost"] = new_validation["payprice"].cumsum()
budget_1 = new_validation[new_validation["total_cost"]/1000<=25000]
statistic["Click"] = [budget_1[budget_1.click==1].click.sum()]
statistic["Spend"] = budget_1.payprice.sum()/1000
statistic["CTR%"] = statistic["Click"]/budget_1.shape[0]*100
statistic["CPC"] = statistic["Spend"]/statistic["Click"]
statistic["CPM"] = statistic["Spend"] /budget_1.shape[0]* 1000

statistic_2=pd.DataFrame()
budget_2 = new_validation[new_validation["total_cost"]/1000<=25000*1/2]
statistic_2["Click"] = [budget_2[budget_2.click==1].click.sum()]
statistic_2["Spend"] = budget_2.payprice.sum()/1000
statistic_2["CTR%"] = statistic_2["Click"]/budget_2.shape[0]*100
statistic_2["CPC"] = statistic_2["Spend"]/statistic_2["Click"]
statistic_2["CPM"] = statistic_2["Spend"] /budget_2.shape[0]* 1000

statistic_4=pd.DataFrame()
budget_4 = new_validation[new_validation["total_cost"]/1000<=25000/4]
statistic_4["Click"] = [budget_4[budget_4.click==1].click.sum()]
statistic_4["Spend"] = budget_4.payprice.sum()/1000
statistic_4["CTR%"] = statistic_4["Click"]/budget_4.shape[0]*100
statistic_4["CPC"] = statistic_4["Spend"]/statistic_4["Click"]
statistic_4["CPM"] = statistic_4["Spend"] /budget_4.shape[0]* 1000

statistic_8=pd.DataFrame()
budget_8 = new_validation[new_validation["total_cost"]/1000<=25000*1/8]
statistic_8["Click"] = [budget_8[budget_8.click==1].click.sum()]
statistic_8["Spend"] = budget_8.payprice.sum()/1000
statistic_8["CTR%"] = statistic_8["Click"]/budget_8.shape[0]*100
statistic_8["CPC"] = statistic_8["Spend"]/statistic_8["Click"]
statistic_8["CPM"] = statistic_8["Spend"] /budget_8.shape[0]* 1000

statistic_16=pd.DataFrame()
budget_16 = new_validation[new_validation["total_cost"]/1000<=25000*1/16]
statistic_16["Click"] = [budget_16[budget_16.click==1].click.sum()]
statistic_16["Spend"] = budget_16.payprice.sum()/1000
statistic_16["CTR%"] = statistic_16["Click"]/budget_16.shape[0]*100
statistic_16["CPC"] = statistic_16["Spend"]/statistic_16["Click"]
statistic_16["CPM"] = statistic_16["Spend"] /budget_16.shape[0]* 1000


statistic_one = statistic.append(statistic_2)
statistic_two = statistic_one.append(statistic_4)
statistic_3 = statistic_two.append(statistic_8)

statistic_full_val_lasso = statistic_3.append(statistic_16)

statistic_full_val_lasso.index = ["full","1/2","1/4 (6250)","1/8","1/16"]
statistic_full_val_lasso

C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPC,CPM
full,97,7381.310,0.055729,76.095979,42.407673
1/2,97,7381.310,0.055729,76.095979,42.407673
1/4 (6250),78,6249.963,0.052917,80.127731,42.401090
1/8,50,3124.988,0.067849,62.499760,42.405493
1/16,26,1562.499,0.070493,60.096115,42.363663


# GBR1

In [37]:
Y_pred_validation_gbr1 = GBR.predict_proba(X_validation)
Y_pred_validation_gbr1 = Y_pred_validation_gbr1 [:,1]

validation["pCTR_gbr1"] = Y_pred_validation_gbr1

# avgCTR = (train[train.click==1].groupby("advertiser").size().values/train.groupby("advertiser").size().values).mean()
avgCTR = Y_pred_validation_gbr1.mean()

param_validation_gbr1= Y_pred_validation_gbr1/avgCTR

basebid = train.bidprice.mean()
# budget = 25000
validation['new_bidprice_gbr1'] = 80*param_validation_gbr1

statistic=pd.DataFrame()
new_validation = validation[validation["new_bidprice_gbr1"]>=validation.payprice]
new_validation["total_cost"] = new_validation["payprice"].cumsum()
budget_1 = new_validation[new_validation["total_cost"]/1000<=25000]
statistic["Click"] = [budget_1[budget_1.click==1].click.sum()]
statistic["Spend"] = budget_1.payprice.sum()/1000
statistic["CTR%"] = statistic["Click"]/budget_1.shape[0]*100
statistic["CPC"] = statistic["Spend"]/statistic["Click"]
statistic["CPM"] = statistic["Spend"] /budget_1.shape[0]* 1000

statistic_2=pd.DataFrame()
budget_2 = new_validation[new_validation["total_cost"]/1000<=25000*1/2]
statistic_2["Click"] = [budget_2[budget_2.click==1].click.sum()]
statistic_2["Spend"] = budget_2.payprice.sum()/1000
statistic_2["CTR%"] = statistic_2["Click"]/budget_2.shape[0]*100
statistic_2["CPC"] = statistic_2["Spend"]/statistic_2["Click"]
statistic_2["CPM"] = statistic_2["Spend"] /budget_2.shape[0]* 1000

statistic_4=pd.DataFrame()
budget_4 = new_validation[new_validation["total_cost"]/1000<=25000/4]
statistic_4["Click"] = [budget_4[budget_4.click==1].click.sum()]
statistic_4["Spend"] = budget_4.payprice.sum()/1000
statistic_4["CTR%"] = statistic_4["Click"]/budget_4.shape[0]*100
statistic_4["CPC"] = statistic_4["Spend"]/statistic_4["Click"]
statistic_4["CPM"] = statistic_4["Spend"] /budget_4.shape[0]* 1000

statistic_8=pd.DataFrame()
budget_8 = new_validation[new_validation["total_cost"]/1000<=25000*1/8]
statistic_8["Click"] = [budget_8[budget_8.click==1].click.sum()]
statistic_8["Spend"] = budget_8.payprice.sum()/1000
statistic_8["CTR%"] = statistic_8["Click"]/budget_8.shape[0]*100
statistic_8["CPC"] = statistic_8["Spend"]/statistic_8["Click"]
statistic_8["CPM"] = statistic_8["Spend"] /budget_8.shape[0]* 1000

statistic_16=pd.DataFrame()
budget_16 = new_validation[new_validation["total_cost"]/1000<=25000*1/16]
statistic_16["Click"] = [budget_16[budget_16.click==1].click.sum()]
statistic_16["Spend"] = budget_16.payprice.sum()/1000
statistic_16["CTR%"] = statistic_16["Click"]/budget_16.shape[0]*100
statistic_16["CPC"] = statistic_16["Spend"]/statistic_16["Click"]
statistic_16["CPM"] = statistic_16["Spend"] /budget_16.shape[0]* 1000


statistic_one = statistic.append(statistic_2)
statistic_two = statistic_one.append(statistic_4)
statistic_3 = statistic_two.append(statistic_8)

statistic_full_val_gbr1 = statistic_3.append(statistic_16)

statistic_full_val_gbr1.index = ["full","1/2","1/4 (6250)","1/8","1/16"]
statistic_full_val_gbr1

C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPC,CPM
full,124,7097.049,0.080913,57.234266,46.309969
1/2,124,7097.049,0.080913,57.234266,46.309969
1/4 (6250),104,6249.977,0.076966,60.095933,46.253641
1/8,62,3124.988,0.091392,50.403032,46.064092
1/16,38,1562.453,0.111784,41.117184,45.962611


# Final  comparsion

In [38]:
final = pd.DataFrame()
final["Lasso"] = statistic_full_val_lasso.iloc[2]
final["GBRT"] = statistic_full_val_gbr1.iloc[2]
final["XGBoost"] = statistic_full_val_xgb.iloc[2]
final["Lasso + BGRT + XGBoost "] = statistic_full_val_mix.iloc[2]
final

,Lasso,GBRT,XGBoost,Lasso + BGRT + XGBoost
Click,78.000000,104.000000,104.000000,104.000000
Spend,6249.963000,6249.977000,6249.994000,6249.994000
CTR%,0.052917,0.076966,0.078488,0.078488
CPC,80.127731,60.095933,60.096096,60.096096
CPM,42.401090,46.253641,47.168342,47.168342


In [76]:
Y_pred_validation_xgb = validation["pCTR_xgb"]

# avgCTR = (train[train.click==1].groupby("advertiser").size().values/train.groupby("advertiser").size().values).mean()
avgCTR = Y_pred_validation_xgb.mean()

param_validation_xgb= Y_pred_validation_xgb/avgCTR

validation['new_bidprice_xgb'] = 70*param_validation_xgb

statistic=pd.DataFrame()
new_validation = validation[validation["new_bidprice_xgb"]>=validation.payprice]
new_validation["total_Spend"] = new_validation["payprice"].cumsum()
new_validation = new_validation[new_validation["total_Spend"]/1000<=25000/4]

statistic = pd.DataFrame()
Imps = new_validation.groupby("advertiser").size().values
statistic["Click"] = new_validation.groupby("advertiser").click.sum().values
statistic["Spend"] = new_validation.groupby("advertiser").payprice.sum().values/1000
statistic["CTR%"] = statistic["Click"]/Imps * 100
statistic["CPM"] = statistic["Spend"]/Imps * 1000 
statistic["CPC"] = statistic["Spend"]/statistic["Click"].values 
index = new_validation.groupby('advertiser').size().index.values
statistic.set_index(index,inplace=True)

Clicks = statistic["Click"].sum()
Cost = statistic["Spend"].sum()
CTR = Clicks / new_validation.shape[0] * 100
CPM = Cost / new_validation.shape[0] * 1000
eCPC = Cost / Clicks

statistic_t = statistic.T
statistic_t.insert(9,"Total",[Clicks,Cost,CTR,CPM,eCPC])
statistic = statistic_t.T

statistic

C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPM,CPC
1458,25.0,1759.118,0.065336,45.973186,70.364720
2259,1.0,76.652,0.029833,22.867542,76.652000
2261,0.0,62.451,0.000000,17.439542,inf
2821,7.0,388.399,0.057523,31.917084,55.485571
2997,24.0,354.889,0.409137,60.499318,14.787042
3358,13.0,631.292,0.101404,49.242746,48.560923
3386,16.0,1157.875,0.063801,46.170947,72.367188
3427,24.0,1098.950,0.107599,49.269222,45.789583
3476,6.0,563.695,0.045704,42.938376,93.949167
Total,116.0,6093.321,0.084942,44.619121,52.528629


In [71]:
Y_pred_validation_Lasso = validation["pCTR_Lasso"]

# avgCTR = (train[train.click==1].groupby("advertiser").size().values/train.groupby("advertiser").size().values).mean()
avgCTR = Y_pred_validation_Lasso.mean()

param_validation_Lasso= Y_pred_validation_Lasso/avgCTR

validation['new_bidprice_Lasso'] = 73.9*param_validation_Lasso

statistic=pd.DataFrame()
new_validation = validation[validation["new_bidprice_Lasso"]>=validation.payprice]
new_validation["total_Spend"] = new_validation["payprice"].cumsum()
new_validation = new_validation[new_validation["total_Spend"]/1000<=25000/4]

statistic = pd.DataFrame()
Imps = new_validation.groupby("advertiser").size().values
statistic["Click"] = new_validation.groupby("advertiser").click.sum().values
statistic["Spend"] = new_validation.groupby("advertiser").payprice.sum().values/1000
statistic["CTR%"] = statistic["Click"]/Imps * 100
statistic["CPM"] = statistic["Spend"]/Imps * 1000 
statistic["CPC"] = statistic["Spend"]/statistic["Click"].values 
index = new_validation.groupby('advertiser').size().index.values
statistic.set_index(index,inplace=True)

Clicks = statistic["Click"].sum()
Cost = statistic["Spend"].sum()
CTR = Clicks / new_validation.shape[0] * 100
CPM = Cost / new_validation.shape[0] * 1000
eCPC = Cost / Clicks

statistic_t = statistic.T
statistic_t.insert(9,"Total",[Clicks,Cost,CTR,CPM,eCPC])
statistic = statistic_t.T

statistic

C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPM,CPC
1458,17.0,1107.645,0.055588,36.218854,65.155588
2259,3.0,297.237,0.036123,35.790126,99.079000
2261,2.0,237.898,0.027933,33.225978,118.949000
2821,6.0,420.684,0.045690,32.035029,70.114000
2997,21.0,289.506,0.376479,51.901398,13.786000
3358,3.0,480.768,0.023223,37.216907,160.256000
3386,11.0,1063.259,0.038272,36.993216,96.659909
3427,8.0,927.861,0.033957,39.384566,115.982625
3476,6.0,691.146,0.032432,37.359243,115.191000
Total,77.0,5516.004,0.051860,37.150812,71.636416


In [41]:
Y_pred_validation_gbr1 = validation["pCTR_gbr1"]

# avgCTR = (train[train.click==1].groupby("advertiser").size().values/train.groupby("advertiser").size().values).mean()
avgCTR = Y_pred_validation_gbr1.mean()

param_validation_gbr1= Y_pred_validation_gbr1/avgCTR

validation['new_bidprice_gbr1'] = 70*param_validation_gbr1

statistic=pd.DataFrame()
new_validation = validation[validation["new_bidprice_gbr1"]>=validation.payprice]
new_validation["total_Spend"] = new_validation["payprice"].cumsum()
new_validation = new_validation[new_validation["total_Spend"]/1000<=25000/4]

statistic = pd.DataFrame()
Imps = new_validation.groupby("advertiser").size().values
statistic["Click"] = new_validation.groupby("advertiser").click.sum().values
statistic["Spend"] = new_validation.groupby("advertiser").payprice.sum().values/1000
statistic["CTR%"] = statistic["Click"]/Imps * 100
statistic["CPM"] = statistic["Spend"]/Imps * 1000 
statistic["CPC"] = statistic["Spend"]/statistic["Click"].values 
index = new_validation.groupby('advertiser').size().index.values
statistic.set_index(index,inplace=True)

Clicks = statistic["Click"].sum()
Cost = statistic["Spend"].sum()
CTR = Clicks / new_validation.shape[0] * 100
CPM = Cost / new_validation.shape[0] * 1000
eCPC = Cost / Clicks

statistic_t = statistic.T
statistic_t.insert(9,"Total",[Clicks,Cost,CTR,CPM,eCPC])
statistic = statistic_t.T

statistic

C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPM,CPC
1458,25.0,1722.788,0.065921,45.427381,68.911520
2259,2.0,82.145,0.059207,24.317644,41.072500
2261,0.0,57.435,0.000000,17.227055,inf
2821,6.0,372.264,0.050680,31.443872,62.044000
2997,23.0,331.211,0.399652,57.551868,14.400478
3358,12.0,615.367,0.094959,48.695656,51.280583
3386,19.0,1131.084,0.074355,44.264235,59.530737
3427,23.0,1065.976,0.104745,48.546134,46.346783
3476,3.0,528.030,0.023463,41.297513,176.010000
Total,113.0,5906.300,0.083602,43.697286,52.268142


In [85]:
Y_pred_validation_gbr=0.25*Y_pred_lasso+0.25*Y_pred_XGB+0.5*Y_pred_validation_gbr1
#Y_pred_validation_combine = 0.1 * Y_pred_validation_log + 0.9 * Y_pred_validation_gbr
#Y_pred_validation_combine =0.05*Y_pred_lasso+0*Y_pred_XGB+0.5*Y_pred_validation_gbr1
avg_ctr = Y_pred_validation_combine.mean()
param_validation_combine = Y_pred_validation_combine/avg_ctr

validation["new_bidprice_combine"] = 70 * param_validation_combine

new_validation = validation[validation["new_bidprice_combine"]>=validation.payprice]
new_validation["total_cost"] = new_validation["payprice"].cumsum()

statistic = pd.DataFrame()
Imps = new_validation.groupby("advertiser").size().values
statistic["Click"] = new_validation.groupby("advertiser").click.sum().values
statistic["Spend"] = new_validation.groupby("advertiser").payprice.sum().values/1000
statistic["CTR%"] = statistic["Click"]/Imps * 100
statistic["CPM"] = statistic["Spend"]/Imps * 1000 
statistic["CPC"] = statistic["Spend"]/statistic["Click"].values 
index = new_validation.groupby('advertiser').size().index.values
statistic.set_index(index,inplace=True)

Clicks = statistic["Click"].sum()
Cost = statistic["Spend"].sum()
CTR = Clicks / new_validation.shape[0] * 100
CPM = Cost / new_validation.shape[0] * 1000
eCPC = Cost / Clicks

statistic_t = statistic.T
statistic_t.insert(9,"Total",[Clicks,Cost,CTR,CPM,eCPC])
statistic = statistic_t.T

statistic

C:\Users\rock\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPM,CPC
1458,25.0,1711.946,0.065821,45.072561,68.477840
2259,2.0,101.295,0.046926,23.767011,50.647500
2261,0.0,73.779,0.000000,17.951095,inf
2821,6.0,380.778,0.049587,31.469256,63.463000
2997,23.0,331.050,0.399444,57.493922,14.393478
3358,12.0,600.886,0.094037,47.087689,50.073833
3386,19.0,1161.834,0.070407,43.053213,61.149158
3427,24.0,1060.016,0.107000,47.258850,44.167333
3476,3.0,542.353,0.022227,40.183226,180.784333
Total,114.0,5963.937,0.081495,42.634266,52.315237
